In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
df_identity = pd.read_csv('train_identity.csv')
df_transactions = pd.read_csv('train_transaction.csv')

In [19]:
class Preprocessor:
    
    def __init__(self,df_identity, df_transactions):
        df = df_transactions.join(df_identity,how='left',on='TransactionID',lsuffix='_left', rsuffix='_right')
        df.drop(inplace=True,columns=['TransactionID_left','TransactionID_right'])
        self.X = df.iloc[:,1:]
        self.y = df.iloc[:,0]
    
    def preprocess(self, weights, threshold) -> pd.DataFrame:
        pass


In [ ]:
from sklearn.tree import DecisionTreeClassifier

class Estimator:
    def __init__(self):
        pass

    def get_score(X, y) -> float:
        pass

In [ ]:
class GreyWolfOptimizer:
    
    def __init__(self,estimator: Estimator, preprocessor: Preprocessor):
        self.estimator = estimator
        self.preprocessor = preprocessor

    def run(self ,iterations = 50, threshold = 0.5):

        #Init algo
        

        bar = tqdm(range(iterations))

        for i in bar: #Epochs
            
            # Something

            pass

    def _calculate_cost(self, weights, threshold):
        X = self.preprocessor.preprocess(threshold=threshold)
        y = self.preprocessor.y

        return self.estimator.get_score(X,y)
    